In [4]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as T

In [5]:
# Зачем это? Нормализация данных ускоряет сходимость
transform = T.Compose([T.ToTensor(), T.Normalize((0.5,), (0.5,))])
train = torchvision.datasets.MNIST('.', train=True, transform=transform, download=True)
loader = torch.utils.data.DataLoader(train, batch_size=128, shuffle=True)

In [6]:
# Простая сеть-энкодер
class Encoder(nn.Module):
    def __init__(self, latent_dim=16):
        super().__init__()
        self.net = nn.Sequential(
            nn.Flatten(),
            nn.Linear(28*28, 128),
            nn.ReLU(),
            nn.Linear(128, latent_dim)
        )
    def forward(self, x):
        return self.net(x)

In [7]:
class Decoder(nn.Module):
    def __init__(self, latent_dim=16):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(latent_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 28*28),
            nn.Tanh(),
            nn.Unflatten(1, (1, 28, 28))
        )
    def forward(self, z):
        return self.net(z)

In [24]:
# Модель
latent_dim = 16
encoder = Encoder(latent_dim)
decoder = Decoder(latent_dim)


In [25]:
# Оптимизатор и loss
optimizer = torch.optim.Adam(list(encoder.parameters()) + list(decoder.parameters()), lr=1e-3)
criterion = nn.MSELoss()

In [26]:
# Тренировочный шаг
for x, _ in loader:
    z = encoder(x)
    x_rec = decoder(z)
    loss = criterion(x_rec, x)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    break  # для примера достаточно одного шага

print(f"Примерный loss: {loss.item():.4f}")

Примерный loss: 0.9350
